<a href="https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject_withaugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb](https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/dataset.zip" -d dataset

# Deep Learning Project - People ReID

In [43]:
# import necessary libraries
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import random
random.seed(10)
# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

Cuda available: True
Cuda device count: 1


# Dataset Preprocessing
Make sure to extract the zip into the 'dataset' folder

In [24]:
def setupLabelsDict(annotations_frame):
   
    labels = {}
    index = 0
    for i in list(annotations_frame):
        if(i != "id"):
            for j in range(min(annotations_frame[i]), max(annotations_frame[i])+1):
                labels[f"{i}-{j}"] = index
                index+=1
    return labels

def getTargetEncoding(id, annotations_frame, labels):
    encoding = [0 for _ in range(len(labels))]
    labels_df = annotations_frame.loc[annotations_frame['id'] == id]
    for label, content in labels_df.items():
        if(label != 'id'):
            encoding[labels["%s-%s" % (label, labels_df[label].iloc[0])]] += 1
    return encoding

In [25]:
def convertAnnotationsFrame(annotations_frame, train_dir):
    annotations_frame = pd.read_csv('dataset/annotations_train.csv')

    img_files = [f for f in listdir(train_dir)]

    augmented_annotations_list = [] 
    for entry in annotations_frame.itertuples():
        for i in img_files:
            if(int(entry[1]) == int(i.split("_")[0])):
                img_with_annotation = {
                    "id": i, 
                    "age": entry[2], 
                    "backpack":entry[3],
                    "bag":entry[4],
                    "handbag":entry[5],
                    "clothes":entry[6],
                    "down":entry[7],
                    "up":entry[8],
                    "hair":entry[9],
                    "hat":entry[10],
                    "gender":entry[11],
                    "upblack":entry[12],
                    "upwhite":entry[13],
                    "upred":entry[14],
                    "uppurple":entry[15],
                    "upyellow":entry[16],
                    "upgray":entry[17],
                    "upblue":entry[18],
                    "upgreen":entry[19],
                    "downblack":entry[20],
                    "downwhite":entry[21],
                    "downpink":entry[22],
                    "downpurple":entry[23],
                    "downyellow":entry[24],
                    "downgray":entry[25],
                    "downblue":entry[26],
                    "downgreen":entry[27],
                    "downbrown":entry[28]
                }
                augmented_annotations_list.append(img_with_annotation)

    augmented_annotations_frame = pd.DataFrame(augmented_annotations_list)
    return augmented_annotations_frame


In [26]:
#labels = setupLabelsDict(augmented_annotations_frame)
#print(augmented_annotations_frame.head())
#id = augmented_annotations_frame.iloc[1, 0]
#print(id)
#labels_df = augmented_annotations_frame.loc[augmented_annotations_frame['id'] == id]
#print(labels_df)
#encoding = getTargetEncoding(augmented_annotations_frame.iloc[1, 0],augmented_annotations_frame, labels)
#print(encoding)

In [27]:
#labels = setupLabelsDict(annotations_frame)
#print(len(labels))
#print(labels)
#people_dataset = PeopleDataset(frame_with_labels=augmented_annotations_frame,
#                                    root_dir='./dataset/train',
#                                    labels=labels)
#
#print("Dataset Initialized")
#dataloader = DataLoader(people_dataset, batch_size=1,
#                        shuffle=True, num_workers=0)
#print("DataLoader Initialized")
#print(len(people_dataset))

In [28]:
#for batch_idx, (inputs, targets) in enumerate(dataloader):
#    print(batch_idx, type(inputs), targets)
#    if batch_idx == 3:
#        break

# Network
## Siamese Network

In [29]:
'''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

class Siamese(nn.Module):

    def __init__(self, resnet):
        super(Siamese, self).__init__()
        self.resnet = resnet
        self.resnet.fc = nn.Sequential(nn.Linear(2048, 1024), nn.Sigmoid())
        self.out = nn.Linear(1024, 1)

    def forward_one(self, x):
        x = self.resnet(x)
        #x = x.view(x.size()[0], -1)
        #x = self.liner(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        dis = torch.abs(out1 - out2)
        out = self.out(dis)
        #  return self.sigmoid(out)
        return out

def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  #alexnet = torchvision.models.alexnet(pretrained=True)
  resnet = torchvision.models.resnet50(pretrained=True, progress=False)
  num_features = resnet.fc.in_features
  resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=num_features, out_features=num_classes),
    torch.nn.Sigmoid()
  )
  #print(resnet)

  # get the number of neurons in the penultimate layer
  #in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  #alexnet.classifier[6] = torch.nn.Sequential(
  #  torch.nn.Linear(in_features=in_features, out_features=num_classes),
  #  torch.nn.Sigmoid()
  #)
  return resnet

Cost function

In [30]:
def get_cost_function():
  cost_function = torch.nn.BCEWithLogitsLoss(size_average=True)
  return cost_function

Optimizer

In [31]:
def get_optimizer(net, lr):
  optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.5, 0.999))
  return optimizer

In [53]:
def test(net, data_loader, cost_function, num_classes, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (img1, img2, targets) in enumerate(data_loader):
      # Load data into GPU
      img1 = img1.to(device)
      img2 = img2.to(device)
      targets = targets.to(torch.float32) #converting to float for BCELoss
      targets = targets.to(device)
      #print(inputs.size())
      #print(input)
      #print(targets.size())
      #print(targets)
        
      # Forward pass
      outputs = net.forward(img1, img2)
      #print(outputs)
      # Apply the loss
      outputs = torch.squeeze(outputs,1)
      #print(outputs.size(), targets.size())
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=img1.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      predicted = torch.round(outputs)
      cumulative_accuracy += predicted.eq(targets).sum().item()/num_classes

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net,data_loader,optimizer,cost_function, num_classes, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (img1, img2, targets) in enumerate(data_loader):
    # Load data into GPU
    img1 = img1.to(device)
    img2 = img2.to(device)
    targets = targets.to(torch.float32) #converting to float for BCELoss
    targets = targets.to(device)

    # Forward pass
    outputs = net.forward(img1, img2)

    # Apply the loss
    outputs = torch.squeeze(outputs,1)
    #print(outputs.size(), targets.size())
    loss = cost_function(outputs,targets)
      
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Reset the optimizer
    optimizer.zero_grad()

    # Better print something, no?
    samples+=img1.shape[0]
    cumulative_loss += loss.item()
    predicted = torch.round(outputs)
    cumulative_accuracy += predicted.eq(targets).sum().item()/num_classes

  return cumulative_loss/samples, cumulative_accuracy/samples*100

In [46]:
class PeopleTrainDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, X1, X2, Y, root_dir, transform):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.X1 = X1
        self.X2 = X2
        self.Y = Y
        self.root_dir = root_dir

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name1 = self.X1[idx]
        img_name2 = self.X2[idx]

        image1 = Image.open("%s/%s" % (self.root_dir, img_name1))
        image2 = Image.open("%s/%s" % (self.root_dir, img_name2))
        if self.transform != None:
          image1 = self.transform(image1)
          image2 = self.transform(image2)
        else:
          image1 = T.ToTensor()(image1)
          image2 = T.ToTensor()(image2)
        image1 = F.interpolate(image1, size=128)  
        image2 = F.interpolate(image2, size=128)  

        sample = (image1, image2, self.Y[idx])
        return sample

class PeopleTestDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.img_files = [f for f in listdir(root_dir)]
        
    def __len__(self):
        return len(self.annotations_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        image = io.imread(self.img_files[idx])
        image = T.ToTensor()(image)
        image = F.interpolate(image, size=128)  
        return image

In [47]:
def split_training_data(root_dir):
  full_dataset_X1 = []
  full_dataset_X2 = []
  full_dataset_Y = []
  img_files = [f for f in listdir(root_dir)]
  random.shuffle(img_files)
  if(len(img_files) % 2 != 0):
    img_files = img_files[:-1]

  for idx, img in enumerate(img_files):
    if(idx % 2 == 0):
      full_dataset_X1.append(img)
    if(idx % 2 != 0):
      full_dataset_X2.append(img)
      if (int(full_dataset_X1[-1].split("_")[0]) == int(full_dataset_X2[-1].split("_")[0])):
        full_dataset_Y.append(torch.tensor(1))
      else:
        full_dataset_Y.append(torch.tensor(0))

  print(len(full_dataset_X1), len(full_dataset_X2), len(full_dataset_Y))
  val_X1 = []
  val_X2 = []
  val_Y = []
  train_X1 = []
  train_X2 = []
  train_Y = []
  for idx, _ in enumerate(full_dataset_Y):
    if (idx <= len(full_dataset_Y)*0.2):
      val_X1.append(full_dataset_X1[idx])
      val_X2.append(full_dataset_X2[idx])
      val_Y.append(full_dataset_Y[idx])
    else:
      train_X1.append(full_dataset_X1[idx])
      train_X2.append(full_dataset_X2[idx])
      train_Y.append(full_dataset_Y[idx])

  return train_X1, train_X2, train_Y, val_X1, val_X2, val_Y

def get_data(batch_size, img_root, test_batch_size=256):
  
  # Prepare data transformations and then combine them sequentially
  # transform = list()
  # transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
  # transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
  # transform = T.Compose(transform)                          # Composes the above transformations into one.
  torchvision_transform = T.Compose([
      T.Resize((256, 256)), 
      T.RandomCrop(224),
      T.RandomHorizontalFlip(),
      T.ToTensor(),
      T.Normalize(
          mean=[0.485, 0.456, 0.406],
          std=[0.229, 0.224, 0.225],
      )
  ])

  # Get splitted data
  train_X1, train_X2, train_Y, val_X1, val_X2, val_Y = split_training_data(root_dir="%s/train" % (img_root))

  # Load data
  training_data = PeopleTrainDataset(X1=train_X1,
                                     X2=train_X2,
                                     Y=train_Y,
                                     root_dir="%s/train" % (img_root),
                                     transform=None)
  
  validation_data = PeopleTrainDataset(X1=val_X1,
                                     X2=val_X2,
                                     Y=val_Y,
                                     root_dir="%s/train" % (img_root),
                                     transform=None)
  
  #test_data = PeopleTestDataset(root_dir="%s/test" % (img_root))

  #print("Dataset Initialized")
  #dataloader = DataLoader(people_dataset, batch_size=,
  #                        shuffle=True, num_workers=0)
  #print("DataLoader Initialized")
  #print(len(people_dataset))
  #full_training_data = torchvision.datasets.MNIST('./dataset', train=True, transform=transform, download=True) 
  #test_data = torchvision.datasets.MNIST('./dataset', train=False, transform=transform, download=True) 
  

  # Create train and validation splits
  #num_samples = len(full_training_data)
  #training_samples = int(num_samples*0.5+1)
  #validation_samples = num_samples - training_samples
#
  #training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
  
  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=0)       #before num_workers=4
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=0) #before num_workers=4
  #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=0) #before num_workers=4
  test_loader = None
  
  return train_loader, val_loader, test_loader

In [39]:
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         epochs=5, 
         img_root='./dataset'):
  from torch.utils.tensorboard import SummaryWriter
  writer = SummaryWriter(log_dir="runs/exp2")

  annotations_frame = pd.read_csv("./dataset/annotations_train.csv")
  augmented_annotations_frame = convertAnnotationsFrame(annotations_frame, "%s/train" % (img_root))
  labels = setupLabelsDict(augmented_annotations_frame)

  # Instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(batch_size=batch_size, img_root=img_root)
  
  # Instantiates the model
  net = initialize_alexnet(num_classes=len(labels)).to(device)
  net.load_state_dict(torch.load("./models/resnet50_20epoch.pth"))
  net = Siamese(net)
  net.to(device)
    
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate)
  
  # Instantiates the cost function
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function, num_classes=len(labels))
  val_loss, val_accuracy = test(net, val_loader, cost_function, num_classes=len(labels))
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  log_values(writer, -1, train_loss, train_accuracy, "Train")
  log_values(writer, -1, val_loss, val_accuracy, "Validation")

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function, num_classes=len(labels))
    val_loss, val_accuracy = test(net, val_loader, cost_function, num_classes=len(labels))
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')
    log_values(writer, e, train_loss, train_accuracy, "Train")
    log_values(writer, e, val_loss, val_accuracy, "Validation")
  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function, num_classes=len(labels))
  val_loss, val_accuracy = test(net, val_loader, cost_function, num_classes=len(labels))
  #test_loss, test_accuracy = test(net, test_loader, cost_function)
  log_values(writer, e, train_loss, train_accuracy, "Train")
  log_values(writer, e, val_loss, val_accuracy, "Validation")
  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  # Closes the logger
  writer.close()
  return net

In [41]:
# Free GPU memory
torch.cuda.empty_cache()
import gc
gc.collect()

# clear runs
#! rm -r runs
%load_ext tensorboard
%tensorboard --logdir=runs/exp2/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [54]:
net = main()
torch.save(net.state_dict(), "./resnet_with_augmentation_20epoch")

6494 6494 6494
Before training:
	 Training loss 0.00515, Training accuracy 1.78
	 Validation loss 0.00301, Validation accuracy 1.78
-----------------------------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 8.00 GiB total capacity; 5.65 GiB already allocated; 14.32 MiB free; 5.87 GiB reserved in total by PyTorch)